In [9]:
%%writefile trap_cuda.cu
#include <stdio.h>
#include <cuda_runtime.h>

__host__ __device__ float f(float x)
{
    return 1.0 / (1.0 + x * x);
}

__global__ void trap(float a, float h, int n, float* dev_sum)
{
    int tid = threadIdx.x + blockIdx.x * blockDim.x;
    if (tid > 0 && tid < n) {
        float x = a + tid * h;
        float y = f(x);
        atomicAdd(dev_sum, y);
    }
}

float host_trap(float a, float b, int N)
{
    float h = (b - a) / N;
    int block_size = 512;
    int num_blocks = (N + block_size - 1) / block_size;
    float* dev_sum;

    cudaMalloc((void**)&dev_sum, sizeof(float));
    cudaMemset(dev_sum, 0, sizeof(float));

    trap<<<num_blocks, block_size>>>(a, h, N, dev_sum);

    float sum;
    cudaMemcpy(&sum, dev_sum, sizeof(float), cudaMemcpyDeviceToHost);

    cudaFree(dev_sum);

    sum += (f(a) + f(b)) / 2.0;
    return h * sum;
}

int main()
{
    float a = 0.0;
    float b = 1.0;
    int n = 100000;

    float result = host_trap(a, b, n);
    printf("%.8f\n", result);

    return 0;
}


Overwriting trap_cuda.cu


In [10]:
!nvcc -o trap trap_cuda.cu

In [12]:
!./trap

0.78538483
